In [13]:
import os
import random
import numpy as np
from Bio import SeqIO
from Bio.PDB import PDBParser, MMCIFIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# Function to read sequences from proteinnet7
def read_proteinnet_sequences(file_path):
    sequences = []
    with open(file_path, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            sequences.append(str(record.seq))
    return sequences

In [3]:
def introduce_variation(sequence, num_variants=5, mutation_rate=0.1):
    # Generate variants by introducing mutations
    variants = []
    for _ in range(num_variants):
        mutated_seq = list(sequence)
        for i in range(len(sequence)):
            if random.random() < mutation_rate:
                mutated_seq[i] = random.choice("ACDEFGHIKLMNPQRSTVWY")  # Random AA substitution
        variants.append("".join(mutated_seq))
    return variants

In [4]:
def save_to_mmcif(structure, output_path):
    # Save the modified structure to mmCIF format
    io = MMCIFIO()
    io.set_structure(structure)
    io.save(output_path)

In [5]:
def generate_and_save_structures(variant_seq, template_structure_path, output_dir, variant_index):
    # Parse the template PDB structure
    parser = PDBParser(QUIET=True)
    template_structure = parser.get_structure("template", template_structure_path)
    
    # Save the variant structure in mmCIF format with unique filename
    output_path = os.path.join(output_dir, f"variant_{variant_index}.cif")
    save_to_mmcif(template_structure, output_path)
    print(f"Saved variant {variant_index} structure to {output_path}")

In [6]:
# ML model for protein stability prediction
def extract_sequence_features(sequence):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    features = np.array([sequence.count(aa) / len(sequence) for aa in amino_acids])
    return features

def generate_training_data_from_file(file_path):
    sequences = read_proteinnet_sequences(file_path)
    data = []
    labels = []
    
    for sequence in sequences:
        features = extract_sequence_features(sequence)
        
        # Example stability score as label (replace with actual data if available)
        stability_score = random.uniform(-10, 10)  # Synthetic stability score range
        data.append(features)
        labels.append(stability_score)
    
    return np.array(data), np.array(labels)

def train_stability_model(X, y):
    model = GradientBoostingRegressor()
    model.fit(X, y)
    return model

def predict_stability(model, sequence):
    features = extract_sequence_features(sequence).reshape(1, -1)
    return model.predict(features)[0]

In [7]:
# Read sequences from the proteinnet4 file for training
proteinnet_file = "proteinnet7"
X, y = generate_training_data_from_file(proteinnet_file)

In [8]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Train stability model
stability_model = train_stability_model(X_train, y_train)
y_pred = stability_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Model Test MSE: {mse:.3f}")

Model Test MSE: 33.336


In [10]:
# Generate variants for a specific sequence
# fasta_file = "mecA.fasta"
template_structure_path = "2d45.pdb"
# mecA seq
sequence = "MLTVYGHRGLPSKAPENTIASFKAASEVEGINWLELDVAITKDEQLIIIHEDYLERTTNMSGEITELNYDEIKDASAGSWFGEKFKDEHLPSFDDVVKIANEYNMNLNVELKGITGPNGLALSKSMVKQVEEQLTNLNQNQEELI"
num_variants = 5
variants = introduce_variation(sequence, num_variants=num_variants)

In [14]:
def calculate_protparam_properties(sequence):
    # Validate the sequence to ensure it contains only valid amino acids
    valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
    if not set(sequence).issubset(valid_amino_acids):
        raise ValueError(f"Invalid character found in sequence: {sequence}")
    
    # Analyze the sequence using Bio.SeqUtils.ProtParam
    analysis = ProteinAnalysis(sequence)
    instability_index = analysis.instability_index()
    
    return instability_index

In [15]:
# Predict and display stability for each variant, and save each variant as a structure
output_dir = "./variants"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, variant_seq in enumerate(variants, start=1):  # Start indexing at 1
    # print(variant_seq + "\n")
    
    try:
        # Predict stability score
        stability_score = predict_stability(stability_model, variant_seq)
        instability_score = calculate_protparam_properties(variant_seq)
        print(f"Variant {i}:")
        print(f"    ML Predicted Stability Score = {stability_score:.3f}")
        print(f"    ProteinAnalysis Instability Score = {instability_score:.3f}")
        
        # Generate and save structure in mmCIF format
        generate_and_save_structures(variant_seq, template_structure_path, output_dir, i)
    except ValueError as e:
        print(f"Error processing variant {i}: {e}")

Variant 1:
    ML Predicted Stability Score = 0.001
    ProteinAnalysis Instability Score = 32.071
Saved variant 1 structure to ./variants/variant_1.cif
Variant 2:
    ML Predicted Stability Score = 0.001
    ProteinAnalysis Instability Score = 36.739
Saved variant 2 structure to ./variants/variant_2.cif
Variant 3:
    ML Predicted Stability Score = -0.004
    ProteinAnalysis Instability Score = 23.254
Saved variant 3 structure to ./variants/variant_3.cif
Variant 4:
    ML Predicted Stability Score = -0.006
    ProteinAnalysis Instability Score = 38.064
Saved variant 4 structure to ./variants/variant_4.cif
Variant 5:
    ML Predicted Stability Score = -0.006
    ProteinAnalysis Instability Score = 38.706
Saved variant 5 structure to ./variants/variant_5.cif


In [16]:
! cp 2d45.cif ./variants/

In [17]:
! foldmason easy-msa ./variants/ result output_dir --report-mode 2

Create directory output_dir
easy-msa ./variants/ result output_dir --report-mode 2 

MMseqs Version:             	1.763a428
Path to ProstT5             	
Chain name mode             	0
Write mapping file          	0
Mask b-factor threshold     	0
Coord store mode            	2
Write lookup file           	1
Input format                	0
File Inclusion Regex        	.*
File Exclusion Regex        	^$
Threads                     	8
Verbosity                   	3
Global sequence weighting   	true
Match ratio                 	0.51
Filter MSA                  	1
Select N most diverse seqs  	1000
Minimum score per column    	-20
Gap open cost               	aa:10,nucl:10
Gap extension cost          	aa:1,nucl:1
Mask profile                	1
Pseudo count mode           	0
AA alignment PCA            	1.1
AA alignment PCB            	4.1
3Di alignment PCA           	1.4
3Di alignment PCB           	1.5
AA alignment score bias     	0.6
3Di alignment score bias    	0.6
Input Newick guide tree 